In [3]:
import tensorflow as tf
# %run ./layers.ipynb

In [4]:
def NFM(sparse_field_size, sparse_feat_size, dense_field_size, 
            emb_size=4, dnn_hidden_units=[128, 128]):
    
    sparse_index = tf.keras.layers.Input(sparse_field_size,)
    dense_value = tf.keras.layers.Input(dense_field_size,)
    
    sparse_w = tf.keras.layers.Embedding(sparse_feat_size, 1)(sparse_index)
    sparse_emb = tf.keras.layers.Embedding(sparse_feat_size, emb_size)(sparse_index)
    
    bi_out = BiInteractionPooling()(sparse_emb)
    dnn_inputs = tf.concat([bi_out, dense_value], axis=1)
    
    dnn_out = DNN()(dnn_inputs)
    
    linear_logit = Linear()([sparse_w, dense_value])
    dnn_logit = tf.keras.layers.Dense(1, use_bias=False)(dnn_out)
    output = tf.sigmoid(tf.add(linear_logit, dnn_logit))
    
    model = tf.keras.models.Model(inputs=[sparse_index, dense_value], outputs=output)
    return model